In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy


# Initial Parameters
All_Lost = 0
BasicOptOfferPrice = 35
BasicOptOfferVol = 50
BasicStore = 80
InitAccount = 10000
Max_Demand = 40
MeanDPrice = 100
OptOfferAcceptDecision = 1
OptOfferBaseVolume = 40
RentRate = 200
Ret_Price = 70
ShopStore = 30
STOP_SELL = 0
TransferDecision = 1
TransferRate = 150
TransferVol = 100
Weekends = 4
Qualification = 5
dt = 1
t = 0

# Initial State
Account = InitAccount
lost_arr = [All_Lost]
shop_store_data = [ShopStore]
basic_store_data = [BasicStore]
Demand = round(Max_Demand * ((1 - 1 / (1 + np.exp(-0.05 * (Ret_Price - MeanDPrice)) ))))
RND_Demand = round(Demand * np.random.uniform(0.9, 1.2) / 
    + Demand * np.random.uniform(0.05, 0.3) * (Qualification/100 /
    + (Weekends/7)))
SoldRet = (1 - STOP_SELL) * min(RND_Demand, ShopStore)
Income = Ret_Price * SoldRet
income_arr = [Income]
demand_arr = []

# Simulation Loop
for t in range(1, 201):  # 200 time steps
    # Prompt user to change parameters
    # if input("Do you want to change parameters during modeling? (Y/N)").lower() == 'y':
    #     TransferVol = float(input("Enter TransferVolume: "))
    #     Ret_Price = float(input("Enter Ret_Price: "))

    # Demand Modeling
    Demand = round(Max_Demand * (1 - 1 / (1 + np.exp(-0.05 * (Ret_Price - MeanDPrice)))))
    RND_Demand = round(Demand * np.random.uniform(0.9, 1.2) + Demand * np.random.uniform(0.05, 0.3) * (Qualification/100 + (Weekends/7)))
    demand_arr.append(RND_Demand)
    SoldRet = (1 - STOP_SELL) * min(RND_Demand, ShopStore)
    Selling = SoldRet
    Income = Ret_Price * SoldRet

    # Financial Transactions
    DailySpending = min(RentRate, Account)
    TransSpend = TransferRate if TransferVol > 0 else 0
    VAT = 0.13 * Income

    # Update Account Balance
    Account += dt * (Income - DailySpending - TransSpend - VAT)

    # Generate Random Price and Volume
    BasicPriceRnd = BasicOptOfferVol * np.random.uniform(0.7, 1.3)
    AddPriceByTime = BasicOptOfferPrice * (0.03 * t + 0.01 * t * np.random.uniform(0, 1))
    OfferOnePrice = AddPriceByTime + BasicPriceRnd
    RndOfferVolume = round(OptOfferBaseVolume * np.random.uniform(0.75, 1.25))
    OfferFullPrice = OfferOnePrice * RndOfferVolume
    OfferAcceptPossibility = 1 if Account >= OfferFullPrice else 0
    SmallOptIncom = OfferAcceptPossibility * OptOfferAcceptDecision * RndOfferVolume

    # Goods Transfer
    TransferActualVolue = min(BasicStore, TransferVol * TransferDecision) if Account >= TransferRate else 0
    GoodsTransfer = np.floor(TransferActualVolue)
    Lost = max(0, ShopStore + GoodsTransfer - 100)

    # Update Store Inventories
    BasicStore += dt * (SmallOptIncom - GoodsTransfer)
    ShopStore += dt * (GoodsTransfer - Selling - Lost)
    All_Lost += dt * Lost
    lost_arr.append(All_Lost)

    # Collect Data
    income_arr.append(Income)
    shop_store_data.append(ShopStore)
    basic_store_data.append(BasicStore)

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Plot Store Sizes
fig_store_sizes = make_subplots(rows=1, cols=2, subplot_titles=['Shop Store Size', 'Basic Store Size'])
fig_store_sizes.add_trace(go.Scatter(x=list(range(201)), y=shop_store_data, mode='lines', name='Shop Store Size'), row=1, col=1)
fig_store_sizes.add_trace(go.Scatter(x=list(range(201)), y=basic_store_data, mode='lines', name='Basic Store Size'), row=1, col=2)
fig_store_sizes.update_layout(title_text='Store Sizes Over Time', xaxis_title='Time Step', yaxis_title='Store Size')
fig_store_sizes.show()

# Plot Lost Goods
fig_lost_goods = go.Figure()
fig_lost_goods.add_trace(go.Scatter(x=list(range(201)), y=lost_arr, mode='lines', name='Lost Goods'))
fig_lost_goods.update_layout(title_text='Lost Goods Over Time', xaxis_title='Time Step', yaxis_title='Lost Goods')
fig_lost_goods.show()

# Plot Demand
fig_demand = go.Figure()
fig_demand.add_trace(go.Scatter(x=list(range(200)), y=demand_arr, mode='lines', name='Demand'))
fig_demand.update_layout(title_text='Demand Over Time', xaxis_title='Time Step', yaxis_title='Demand')
fig_demand.show()

# Plot Daily Spending and Income
fig_daily_finances = go.Figure()
fig_daily_finances.add_trace(go.Scatter(x=list(range(201)), y=[min(RentRate, acc) for acc in [InitAccount] + income_arr], mode='lines', name='Daily Spending'))
fig_daily_finances.add_trace(go.Scatter(x=list(range(201)), y=[TransferRate if TransferVol > 0 else 0 for _ in range(201)], mode='lines', name='Transfer Spending'))
fig_daily_finances.add_trace(go.Scatter(x=list(range(201)), y=[0.13 * inc for inc in income_arr], mode='lines', name='VAT'))
fig_daily_finances.add_trace(go.Scatter(x=list(range(201)), y=income_arr, mode='lines', name='Income'))
fig_daily_finances.update_layout(title_text='Daily Finances Over Time', xaxis_title='Time Step', yaxis_title='Amount')
fig_daily_finances.show()